##building and saving model

In [ ]:
 ##download dataset from gdrive 
 import shutil
 shutil.copy2('/content/drive/My Drive/Dataset/Dataset.zip','.')

'./Dataset.zip'

In [ ]:
#extract the dataset
from zipfile import ZipFile
fname='Dataset.zip'
with ZipFile(fname,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
#make folder name dataset and cut the files into it for convinence
shutil.move('covid','dataset')
shutil.move('normal','dataset')

'dataset/normal'

In [ ]:
#now separate train,test and validation data
#first change the name of covid to 0 and normal to 1 manually
import os
import random
os.chdir('dataset')
if os.path.isdir('train/0/') is False:
  os.mkdir('train')
  os.mkdir('valid')
  os.mkdir('test')

  for i in range(0,2):
    shutil.move(f'{i}','train')
    os.mkdir(f'valid/{i}')
    os.mkdir(f'test/{i}')
    #move directory to train and make drictorys in valid and test
    #now we have total 180 images so we take 25 for validation and 25 for test and rest of it for trainnig purpose

    test_samples=random.sample(os.listdir(f'train/{i}'),25)
    for j in test_samples:
      shutil.move(f'train/{i}/{j}',f'test/{i}')

    valid_samples=random.sample(os.listdir(f'train/{i}'),25)
    for j in valid_samples:
      shutil.move(f'train/{i}/{j}',f'valid/{i}') 
    


In [ ]:
os.chdir('../')

In [ ]:
!ls

dataset  Dataset.zip  drive  sample_data


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [ ]:
##now the dataset is completly ready for preprocessing
#now we have only 150 images so we use image datagenerator for making more images out of it
#here we use famous vgg16 filter
#logically we have to apply it only for train but we used same for test and validaton(genarally we only only take preproccessing to it)
train_datagen=image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    width_shift_range=.1,
    height_shift_range=.1,
    zoom_range=.2,
    shear_range=.2,
    horizontal_flip=True,
                                       )

test_datagen=image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    width_shift_range=.1,
    height_shift_range=.1,
    zoom_range=.2,
    shear_range=.2,
    horizontal_flip=True,
                                       )

valid_datagen=image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    width_shift_range=.1,
    height_shift_range=.1,
    zoom_range=.2,
    shear_range=.2,
    horizontal_flip=True,
                                       )

In [ ]:
train_path='dataset/train'
valid_path='dataset/valid'
test_path='dataset/test'

In [ ]:
train_gen=train_datagen.flow_from_directory(
    train_path,
    target_size=(448,448),
    batch_size=32,
)

test_gen=test_datagen.flow_from_directory(
    test_path,
    target_size=(448,448),
    batch_size=32,
)

valid_gen=valid_datagen.flow_from_directory(
    valid_path,
    target_size=(448,448),
    batch_size=32,
)

Found 260 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [ ]:
train_gen.class_indices

{'0': 0, '1': 1}

In [ ]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(448,448,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Flatten())
model.add(Dense(2048,activation='relu'))
model.add(Dropout(.5))

model.add(Dense(512,activation='relu'))
model.add(Dropout(.5))

model.add(Dense(64,activation='relu'))
model.add(Dropout(.5))

model.add(Dense(16,activation='relu'))
model.add(Dropout(.5))

model.add(Dense(2,activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 446, 446, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 444, 444, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 222, 222, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 222, 222, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 64)      0

In [ ]:
model.fit(x=train_gen,validation_data=valid_gen,steps_per_epoch=8,epochs=15,verbose=2)

Epoch 1/15
8/8 - 19s - loss: 64.2611 - accuracy: 0.5439 - val_loss: 0.8328 - val_accuracy: 0.5400
Epoch 2/15
8/8 - 18s - loss: 38.4446 - accuracy: 0.5088 - val_loss: 0.7200 - val_accuracy: 0.5000
Epoch 3/15
8/8 - 18s - loss: 16.3634 - accuracy: 0.4956 - val_loss: 0.6920 - val_accuracy: 0.6000
Epoch 4/15
8/8 - 18s - loss: 8.9785 - accuracy: 0.5088 - val_loss: 0.7193 - val_accuracy: 0.2400
Epoch 5/15
8/8 - 18s - loss: 8.7916 - accuracy: 0.4956 - val_loss: 0.7130 - val_accuracy: 0.1000
Epoch 6/15
8/8 - 18s - loss: 4.4356 - accuracy: 0.5044 - val_loss: 0.7023 - val_accuracy: 0.2000
Epoch 7/15
8/8 - 18s - loss: 3.7336 - accuracy: 0.5195 - val_loss: 0.6997 - val_accuracy: 0.2800
Epoch 8/15
8/8 - 18s - loss: 2.8100 - accuracy: 0.4605 - val_loss: 0.6980 - val_accuracy: 0.2400
Epoch 9/15
8/8 - 18s - loss: 1.8200 - accuracy: 0.4961 - val_loss: 0.6960 - val_accuracy: 0.5200
Epoch 10/15
8/8 - 18s - loss: 1.4196 - accuracy: 0.5175 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 11/15
8/8 - 18s - lo

In [ ]:
model.predict(test_gen)

array([[0.4963607 , 0.50363934],
       [0.49618277, 0.5038172 ],
       [0.49760586, 0.50239414],
       [0.49677995, 0.5032201 ],
       [0.49503353, 0.5049665 ],
       [0.49548402, 0.50451595],
       [0.49695832, 0.5030417 ],
       [0.49579751, 0.5042025 ],
       [0.49580908, 0.5041909 ],
       [0.49552643, 0.50447357],
       [0.4964437 , 0.5035563 ],
       [0.49578434, 0.50421566],
       [0.49778852, 0.50221145],
       [0.4978315 , 0.5021685 ],
       [0.5010125 , 0.49898744],
       [0.5000668 , 0.4999332 ],
       [0.49733117, 0.50266886],
       [0.49620298, 0.50379705],
       [0.49606344, 0.5039365 ],
       [0.49681866, 0.5031813 ],
       [0.4965728 , 0.50342715],
       [0.4959005 , 0.5040994 ],
       [0.49863023, 0.5013698 ],
       [0.49733642, 0.5026636 ],
       [0.49667972, 0.5033203 ],
       [0.49831438, 0.5016856 ],
       [0.49788517, 0.50211483],
       [0.4971767 , 0.5028233 ],
       [0.4966976 , 0.5033024 ],
       [0.49713972, 0.50286025],
       [0.

In [ ]:
##after shwong above outputs the model is not accurate at predictions
##so i tried another model